In [2]:
import sys
sys.path.append('../../src/')

In [3]:
%load_ext autoreload
%autoreload 2

from data import load_data

In [4]:
import os
import copy
import time
import random
import pickle

import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm import tqdm

from sklearn.metrics import f1_score, confusion_matrix
from numpy.random import shuffle
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import torch.autograd.profiler as tprofiler
import torch.utils.data as td

plt.rcParams["savefig.bbox"] = 'tight'

In [5]:
seed = 42

random.seed(seed)

# pytorch RNGs
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
np.random.seed(seed)

In [6]:
data_dir = "../../data"
images_dir = "../../data/resized_processed"

In [7]:
train = pd.read_csv(os.path.join(data_dir,"train.csv"))
val = pd.read_csv(os.path.join(data_dir,"val.csv"))
test = pd.read_csv(os.path.join(data_dir,"test.csv"))

In [8]:
train.head(2)

,Patient ID,label,FilePath
0,2,No Finding,images_001/images/00000002_000.png
1,5,No Finding,images_001/images/00000005_003.png


In [11]:
num_workers = 8
batch_size = 64
image_size=299

In [14]:
aug = T.Compose([T.Resize(size=image_size),
                 T.ToTensor(),
                 T.Normalize(mean = (0, 0, 0),
                              std  = (1, 1, 1))])

train_dataset = datasets.ImageFolder(root=os.path.join(images_dir, "train"),
                                     transform=aug)
val_dataset = datasets.ImageFolder(root=os.path.join(images_dir, "val"),
                                   transform=aug)
test_dataset = datasets.ImageFolder(root=os.path.join(images_dir, "test"),
                                    transform=aug)

data_loader_train = td.DataLoader(train_dataset,
                                  batch_size=batch_size,
                                  shuffle=False,
                                  drop_last=False,
                                  num_workers=num_workers,
                                  pin_memory=True)
data_loader_val = td.DataLoader(val_dataset,
                                batch_size=batch_size,
                                shuffle=False,
                                drop_last=False,
                                num_workers=num_workers,
                                pin_memory=True)
data_loader_test = td.DataLoader(test_dataset,
                                 batch_size=batch_size,
                                 shuffle=False,
                                 drop_last=False,
                                 num_workers=num_workers,
                                 pin_memory=True)

In [ ]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs, labels in tqdm(data_loader_train):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|████████████████████████████████████████████████████████████████████████████████| 611/611 [01:01<00:00,  9.91it/s]


In [ ]:
count = len(train) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.4951, 0.4951, 0.4951])
std:  tensor([0.2896, 0.2896, 0.2896])


In [29]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs, labels in tqdm(data_loader_val):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|████████████████████████████████████████████████████████████████████████████████| 133/133 [00:19<00:00,  6.70it/s]


In [30]:
count = len(val) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.4954, 0.4954, 0.4954])
std:  tensor([0.2893, 0.2893, 0.2893])


In [26]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs, labels in tqdm(data_loader_test):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|████████████████████████████████████████████████████████████████████████████████| 132/132 [00:19<00:00,  6.70it/s]


In [28]:
count = len(test) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.4954, 0.4954, 0.4954])
std:  tensor([0.2893, 0.2893, 0.2893])
